In [6]:
from sklearn.svm import SVR
from sklearn.model_selection import KFold
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import cohen_kappa_score, mean_squared_error, r2_score
import statistics
from keras.layers import Embedding, LSTM, Dense, Dropout, Lambda, Flatten
from keras.models import Sequential, load_model, model_from_config
import keras.backend as K

import numpy as np
import pandas as pd

In [53]:
df = pd.read_csv("../df.csv")
doc2vec50 = pd.read_csv("../dataset/doc2vec_50.csv")
df_train= df[["essay_set", "essay","total_score", "word_count","Mistakes","reading_ease"]]
df_train = doc2vec50
df_train = pd.concat([df_train, df[["essay_set", "word_count","Mistakes","reading_ease"]]], axis = 1, join = "inner")
y = df["total_score"]
X = df_train.drop(['Unnamed: 0'], axis = 1)


              0         1         2         3         4         5         6  \
0     -0.653643  1.226314  1.490607 -0.273509 -0.126667  1.948407 -0.800371   
1      1.788302 -0.599212 -2.108530  0.105541 -0.419768  0.424503  0.306117   
2     -0.024527  0.521169 -0.856823 -0.467126 -0.856623  0.542416 -1.578355   
3      2.232087  0.245010 -0.081397  2.165611 -1.296768  0.386925 -1.609302   
4     -0.532192 -0.046631 -1.994325  0.103210 -1.187608  0.603498  0.119099   
...         ...       ...       ...       ...       ...       ...       ...   
12971  1.006991 -0.286482  1.342189 -0.003134  0.760130 -0.679358 -1.243566   
12972  0.111378  0.336479  0.752918 -0.181376  0.736306 -1.199180 -0.596781   
12973  1.191366  0.973212 -0.547442 -1.165032 -1.202222 -1.381345 -0.519082   
12974  0.418220  1.790897  0.547983 -1.530168  0.991451  0.504360 -1.435510   
12975 -0.080305  1.266496  0.187342 -0.565856  0.896607  0.895382 -0.369216   

              7         8         9  ...        44 

In [54]:
def get_model(x_train_shape):
    """Define the model."""
    model = Sequential()
    model.add(LSTM(50, dropout=0.2, recurrent_dropout=0.4, input_shape=(1, 54) , return_sequences=True))
    model.add(LSTM(50, recurrent_dropout=0.4,  return_sequences = True))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='relu'))

    model.compile(loss='mean_squared_error', optimizer='rmsprop', metrics=['mae'])
    model.summary()

    return model

In [59]:
cv = KFold(n_splits = 5, shuffle = True)
results = []
r2s = []
mses = []
y_pred_list = []
count = 1
for traincv, testcv in cv.split(X):
    print("\n--------Fold {}--------\n".format(count))
    
    X_test, X_train, y_test, y_train = X.iloc[testcv], X.iloc[traincv], y.iloc[testcv], y.iloc[traincv]
    X_train = np.array(X_train)
    X_test = np.array(X_test)
    X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
    X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))
    lstm_model = get_model(X_train.shape)
    lstm_model.fit(X_train, y_train, validation_split=0.2, epochs=2, batch_size=32)
    #lstm_model.load_weights('./model_weights/final_lstm.h5')
    
    
    y_pred = lstm_model.predict(X_test)
    
    # Save any one of the 5 models.
    if count == 5:
         lstm_model.save_weights('final_lstm.h5')
    
    # Round y_pred to the nearest integer.
    y_pred = np.around(y_pred)
    
    for i in y_pred_temp:
        y_pred.append(int(round(i)))
    y_test_new = []
    for  i in list(y_test.array):
        y_test_new.append(int(i))
    # Evaluate the model on the evaluation metric. "Quadratic mean averaged Kappa"
    result = cohen_kappa_score(y_test.values,y_pred,weights='quadratic')
    print("Kappa Score: {}".format(result))
    results.append(result)

    count += 1


--------Fold 1--------

Model: "sequential_23"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_43 (LSTM)               (None, 1, 50)             21000     
_________________________________________________________________
lstm_44 (LSTM)               (None, 1, 50)             20200     
_________________________________________________________________
dropout_20 (Dropout)         (None, 1, 50)             0         
_________________________________________________________________
dense_20 (Dense)             (None, 1, 1)              51        
Total params: 41,251
Trainable params: 41,251
Non-trainable params: 0
_________________________________________________________________
Epoch 1/2
260/260 [==============================] - 2s 8ms/step - loss: 4.4513 - mae: 1.4165 - val_loss: 437.1030 - val_mae: 17.5144
Epoch 2/2
260/260 [==============================] - 1s 4ms/step - loss: 1.1520 - mae: 0

NameError: name 'y_pred_temp' is not defined

[[-6.53643310e-01  1.22631383e+00  1.49060738e+00 ...  3.38000000e+02
   2.00000000e+01  7.40200000e+01]
 [ 1.78830183e+00 -5.99211633e-01 -2.10853004e+00 ...  4.19000000e+02
   3.30000000e+01  6.70800000e+01]
 [-2.45267693e-02  5.21169484e-01 -8.56822908e-01 ...  2.79000000e+02
   2.00000000e+01  6.82000000e+01]
 ...
 [ 1.19136560e+00  9.73212183e-01 -5.47442257e-01 ...  8.18000000e+02
   8.20000000e+01  6.92500000e+01]
 [ 4.18220490e-01  1.79089689e+00  5.47983468e-01 ...  5.94000000e+02
   3.30000000e+01  7.20500000e+01]
 [-8.03048015e-02  1.26649606e+00  1.87341690e-01 ...  4.68000000e+02
   1.10000000e+01  7.20500000e+01]]
